In [1]:
#  Install required packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5

In [2]:
import os, sys, platform
from google.colab import userdata
from datetime import datetime

# LangChain + Gemini
import langchain, langchain_core
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [3]:
#  Set your Gemini API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

In [4]:
# Recommended for demos: fast & economical
model_name = "gemini-2.0-flash"

llm = ChatGoogleGenerativeAI(
    model=model_name,
    convert_system_message_to_human=True,  # smoother behavior with system prompts
    temperature=0.7,
)
print(f" Ready with model: {model_name}")

 Ready with model: gemini-2.0-flash


# Simple sequential chain (Ideas → Summary)

In [5]:
# Step 1: generate ideas
idea_prompt = PromptTemplate.from_template(
    "Generate 3 creative ideas about: {topic}. Return them as a numbered list."
)

# Step 2: summarize the ideas
summary_prompt = PromptTemplate.from_template(
    "Summarize the ideas below in 2 friendly sentences for a beginner:\n\n{ideas}"
)


In [6]:
# Chain

idea_chain = idea_prompt | llm | StrOutputParser()

In [7]:
# Compose a chain where the output of idea_chain is fed into the summary prompt
sequential_chain = {
    "ideas": idea_chain,
    "topic": RunnablePassthrough()
} | summary_prompt | llm | StrOutputParser()

In [8]:
topic = "using AI to improve school homework feedback"
Creative_ideas = idea_chain.invoke({"topic": topic})
summary = sequential_chain.invoke({"topic": topic})

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [9]:
print(" Topic:", topic)
print(" Creative Ideas : ", Creative_ideas)
print(" Summary:\n", summary)

 Topic: using AI to improve school homework feedback
 Creative Ideas :  Here are 3 creative ideas about using AI to improve school homework feedback:

1.  **Personalized Feedback Narrative Generation with "Character" Tutoring:** Imagine an AI system that analyzes student homework and generates feedback in the form of a personalized story or narrative. Instead of just saying "You need to elaborate on your thesis statement," the AI might present a scenario: "Professor Owl, your AI tutor, has noticed a fascinating seed of an idea in your opening paragraph! However, it's a bit like a hidden acorn – it needs more sunlight (details and evidence) to grow into a strong, convincing tree. Let's explore how you can expand on this concept..." This approach leverages different "character" tutors (e.g., a detective for persuasive writing, a builder for math problems) each with a unique voice and learning style to cater to individual student preferences. The AI would also track which character and na